In [5]:
# %%
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base, Session

SQLALCHEMY_DATABASE_URL = 'sqlite:///./api_data.db'

engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={'check_same_thread': False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# %% [markdown]
# ## Models

# %%
#models
from sqlalchemy import Column, Integer, String, Text, Float, Boolean, DateTime
import datetime

class ElectricityPrice(Base):
    __tablename__ = 'electricity_prices'
    id = Column(Integer, primary_key=True, index=True)
    city = Column(String, index=True)
    state = Column(String, index=True)
    date_query = Column(String, index=True)
    region = Column(String, index=True)
    NOK_per_kWh = Column(Float)
    EUR_per_kWh = Column(Float)
    EXR = Column(Float)
    time_start = Column(String)
    time_end = Column(String)
    timestamp = Column(DateTime, default=datetime.datetime.now(datetime.UTC))

class WeatherData(Base):
    __tablename__ = 'weather_data'

    id = Column(Integer, primary_key=True, index=True)
    city = Column(String, index=True)
    state = Column(String, index=True)
    date_query = Column(String)
    temperature = Column(Float)
    time_start = Column(Text)
    time_end = Column(Text)
    timestamp = Column(DateTime, default=datetime.datetime.now(datetime.UTC))

# %%
# schemas.py
from pydantic import BaseModel, Field
from typing import List

class PriceData(BaseModel):
    NOK_per_kWh: float
    EUR_per_kWh: float
    EXR: float
    time_start: str
    time_end: str

class ElectricityPriceRequest(BaseModel):
    city: str = Field(description="City")
    state: str = Field(description="State or region")
    date_query: str = Field(description="Date query (e.g., 'today', 'yesterday', 'last 10 days', 'last 30 days', 'last three months', or 'YYYY-MM-DD/YYYY-MM-DD')")

# Pydantic models
class WeatherDataSchema(BaseModel):
    temperature: float
    time_start: str
    time_end: str

class WeatherRequest(BaseModel):
    city: str = Field(description="City")
    state: str = Field(description="State or region")
    date_query: str = Field(description="Date query (e.g., 'today', 'yesterday', 'last 10 days', 'last 30 days', 'last three months', or 'YYYY-MM-DD/YYYY-MM-DD')")

# %%
# Function to get a database session
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# Function to store data

def store_data(db: Session, model, data: list):
    for item in data:
        db_item = model(**item)
        db.add(db_item)
    db.commit()

# %% [markdown]
# ## Electricricty Price API


# Create the database tables
Base.metadata.create_all(bind=engine)


from sqlalchemy.orm import Session
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from datetime import datetime, timedelta
import requests
import json
import requests_cache
from retry_requests import retry
import pandas as pd
import openmeteo_requests

# Assuming the database and models are already set up as in your original code.

# Tool to fetch electricity prices
class ElectricityPriceTool:
    REGIONS = {
        "NO1": ("Oslo", (59.9139, 10.7522)),
        "NO2": ("Kristiansand", (58.1467, 7.9956)),
        "NO3": ("Trondheim", (63.4305, 10.3951)),
        "NO4": ("Tromsø", (69.6492, 18.9553)),
        "NO5": ("Bergen", (60.3928, 5.3221)),
    }

    def __init__(self, db: Session):
        self.db = db

    def get_coordinates(self, city_name: str, state: str) -> Optional[tuple]:
        geolocator = Nominatim(user_agent="electricity_price_api")
        location = geolocator.geocode(f"{city_name}, {state}")
        if location:
            return location.latitude, location.longitude
        return None

    def find_nearest_region(self, lat: float, lon: float) -> Optional[str]:
        min_distance = float("inf")
        nearest_region = None
        for region, (city, coordinates) in self.REGIONS.items():
            distance = geodesic((lat, lon), coordinates).kilometers
            if distance < min_distance:
                min_distance = distance
                nearest_region = region
        return nearest_region

    def fetch_electricity_prices(self, year: str, month: str, day: str, region: str) -> list:
        url = f"https://www.hvakosterstrommen.no/api/v1/prices/{year}/{month}-{day}_{region}.json"
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        return {"error": f"Error fetching data: {response.status_code}"}

    def store_prices(self, request: Dict[str, Any], region: str, prices: list):
        data = [
            {
                "city": request['city'],
                "state": request['state'],
                "date_query": request['date_query'],
                "region": region,
                "NOK_per_kWh": price["NOK_per_kWh"],
                "EUR_per_kWh": price["EUR_per_kWh"],
                "EXR": price["EXR"],
                "time_start": price["time_start"],
                "time_end": price["time_end"]
            }
            for price in prices
        ]
        store_data(self.db, ElectricityPrice, data)

    def get_electricity_prices(self, request: Dict[str, Any]) -> List[Dict[str, Any]]:
        coordinates = self.get_coordinates(request['city'], request['state'])
        if not coordinates:
            return {"error": "City not found"}

        lat, lon = coordinates
        start_date, end_date = parse_date_query(request['date_query'])
        nearest_region = self.find_nearest_region(lat, lon)
        if nearest_region:
            prices = self.fetch_electricity_prices(start_date[:4], start_date[5:7], start_date[8:10], nearest_region)
            if isinstance(prices, list):
                self.store_prices(request, nearest_region, prices)
                return prices
            return prices  # Return the error message
        return {"error": "Could not determine the nearest region."}

# Tool to fetch and store weather data
class WeatherDataTool:

    def __init__(self, db: Session):
        self.db = db

    def store_weather_data(self, request: Dict[str, Any], observations: list):
        data = [
            {
                "city": request['city'],
                "state": request['state'],
                "date_query": request['date_query'],
                "temperature": observation['temperature'],
                "time_start": observation['time_start'],
                "time_end": observation['time_end'],
            }
            for observation in observations
        ]
        store_data(self.db, WeatherData, data)

    def get_weather_data(self, request: Dict[str, Any]) -> List[Dict[str, Any]]:
        coordinates = ElectricityPriceTool(self.db).get_coordinates(request['city'], request['state'])
        if not coordinates:
            return {"error": "Could not find coordinates for the specified city and state."}

        latitude, longitude = coordinates
        start_date, end_date = parse_date_query(request['date_query'])

        # Setup the Open-Meteo API client with cache and retry on error
        cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
        retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
        openmeteo = openmeteo_requests.Client(session=retry_session)

        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "start_date": start_date,
            "end_date": end_date,
            "hourly": "temperature_2m"
        }
        responses = openmeteo.weather_api(url, params=params)

        # Process first location
        response = responses[0]

        # Process hourly data
        hourly = response.Hourly()
        hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

        hourly_dates = pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ).strftime('%Y-%m-%dT%H:%M:%S%z')

        # Create a list of dictionaries for each observation
        observations = []
        for i in range(len(hourly_dates) - 1):
            observations.append({
                "temperature": float(hourly_temperature_2m[i]),
                "time_start": hourly_dates[i],
                "time_end": hourly_dates[i + 1]
            })

        self.store_weather_data(request, observations)

        return observations

# Utility function to parse date queries
def parse_date_query(date_query: str) -> (str, str):
    today = datetime.utcnow().date()
    if date_query.lower() == 'today':
        start_date = end_date = today
    elif date_query.lower() == 'yesterday':
        start_date = end_date = today - timedelta(days=1)
    elif 'last' in date_query.lower():
        if "10 days" in date_query.lower():
            start_date = today - timedelta(10)
        elif "30 days" in date_query.lower():
            start_date = today - timedelta(30)
        elif "three months" in date_query.lower():
            start_date = today - timedelta(90)
        else:
            raise ValueError("Invalid date query")
        end_date = today
    elif '/' in date_query:
        start_date, end_date = date_query.split('/')
        start_date = datetime.strptime(start_date, '%Y-%m-%d').date()
        end_date = datetime.strptime(end_date, '%Y-%m-%d').date()
    else:
        raise ValueError("Invalid date_query format.")
    return start_date.isoformat(), end_date.isoformat()

# Example usage for LangChain tools:
def main():
    db = SessionLocal()

    # Electricity Price Tool
    electricity_tool = ElectricityPriceTool(db)
    electricity_request = {
        "city": "Lillehammer",
        "state": "Oslo",
        "date_query": "2024-01-01/2024-01-01"
    }
    electricity_data = electricity_tool.get_electricity_prices(electricity_request)
    print(json.dumps(electricity_data, indent=4))

    # Weather Data Tool
    weather_tool = WeatherDataTool(db)
    weather_request = {
        "city": "Lillehammer",
        "state": "",
        "date_query": "2024-07-23/2024-08-06"
    }
    weather_data = weather_tool.get_weather_data(weather_request)
    print(json.dumps(weather_data, indent=4))

if __name__ == "__main__":
    main()

In [6]:
from sqlalchemy.orm import Session
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from datetime import datetime, timedelta
import requests
import json
import requests_cache
from retry_requests import retry
import pandas as pd
import openmeteo_requests

# Assuming the database and models are already set up as in your original code.

# Tool to fetch electricity prices
class ElectricityPriceTool:
    REGIONS = {
        "NO1": ("Oslo", (59.9139, 10.7522)),
        "NO2": ("Kristiansand", (58.1467, 7.9956)),
        "NO3": ("Trondheim", (63.4305, 10.3951)),
        "NO4": ("Tromsø", (69.6492, 18.9553)),
        "NO5": ("Bergen", (60.3928, 5.3221)),
    }

    def __init__(self, db: Session):
        self.db = db

    def get_coordinates(self, city_name: str, state: str) -> Optional[tuple]:
        geolocator = Nominatim(user_agent="electricity_price_api")
        location = geolocator.geocode(f"{city_name}, {state}")
        if location:
            return location.latitude, location.longitude
        return None

    def find_nearest_region(self, lat: float, lon: float) -> Optional[str]:
        min_distance = float("inf")
        nearest_region = None
        for region, (city, coordinates) in self.REGIONS.items():
            distance = geodesic((lat, lon), coordinates).kilometers
            if distance < min_distance:
                min_distance = distance
                nearest_region = region
        return nearest_region

    def fetch_electricity_prices(self, year: str, month: str, day: str, region: str) -> list:
        url = f"https://www.hvakosterstrommen.no/api/v1/prices/{year}/{month}-{day}_{region}.json"
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        return {"error": f"Error fetching data: {response.status_code}"}

    def store_prices(self, request: Dict[str, Any], region: str, prices: list):
        data = [
            {
                "city": request['city'],
                "state": request['state'],
                "date_query": request['date_query'],
                "region": region,
                "NOK_per_kWh": price["NOK_per_kWh"],
                "EUR_per_kWh": price["EUR_per_kWh"],
                "EXR": price["EXR"],
                "time_start": price["time_start"],
                "time_end": price["time_end"]
            }
            for price in prices
        ]
        store_data(self.db, ElectricityPrice, data)

    def get_electricity_prices(self, request: Dict[str, Any]) -> List[Dict[str, Any]]:
        coordinates = self.get_coordinates(request['city'], request['state'])
        if not coordinates:
            return {"error": "City not found"}

        lat, lon = coordinates
        start_date, end_date = parse_date_query(request['date_query'])
        nearest_region = self.find_nearest_region(lat, lon)
        if nearest_region:
            prices = self.fetch_electricity_prices(start_date[:4], start_date[5:7], start_date[8:10], nearest_region)
            if isinstance(prices, list):
                self.store_prices(request, nearest_region, prices)
                return prices
            return prices  # Return the error message
        return {"error": "Could not determine the nearest region."}

# Tool to fetch and store weather data
class WeatherDataTool:

    def __init__(self, db: Session):
        self.db = db

    def store_weather_data(self, request: Dict[str, Any], observations: list):
        data = [
            {
                "city": request['city'],
                "state": request['state'],
                "date_query": request['date_query'],
                "temperature": observation['temperature'],
                "time_start": observation['time_start'],
                "time_end": observation['time_end'],
            }
            for observation in observations
        ]
        store_data(self.db, WeatherData, data)

    def get_weather_data(self, request: Dict[str, Any]) -> List[Dict[str, Any]]:
        coordinates = ElectricityPriceTool(self.db).get_coordinates(request['city'], request['state'])
        if not coordinates:
            return {"error": "Could not find coordinates for the specified city and state."}

        latitude, longitude = coordinates
        start_date, end_date = parse_date_query(request['date_query'])

        # Setup the Open-Meteo API client with cache and retry on error
        cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
        retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
        openmeteo = openmeteo_requests.Client(session=retry_session)

        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "start_date": start_date,
            "end_date": end_date,
            "hourly": "temperature_2m"
        }
        responses = openmeteo.weather_api(url, params=params)

        # Process first location
        response = responses[0]

        # Process hourly data
        hourly = response.Hourly()
        hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

        hourly_dates = pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ).strftime('%Y-%m-%dT%H:%M:%S%z')

        # Create a list of dictionaries for each observation
        observations = []
        for i in range(len(hourly_dates) - 1):
            observations.append({
                "temperature": float(hourly_temperature_2m[i]),
                "time_start": hourly_dates[i],
                "time_end": hourly_dates[i + 1]
            })

        self.store_weather_data(request, observations)

        return observations

# Utility function to parse date queries
def parse_date_query(date_query: str) -> (str, str):
    today = datetime.utcnow().date()
    if date_query.lower() == 'today':
        start_date = end_date = today
    elif date_query.lower() == 'yesterday':
        start_date = end_date = today - timedelta(days=1)
    elif 'last' in date_query.lower():
        if "10 days" in date_query.lower():
            start_date = today - timedelta(10)
        elif "30 days" in date_query.lower():
            start_date = today - timedelta(30)
        elif "three months" in date_query.lower():
            start_date = today - timedelta(90)
        else:
            raise ValueError("Invalid date query")
        end_date = today
    elif '/' in date_query:
        start_date, end_date = date_query.split('/')
        start_date = datetime.strptime(start_date, '%Y-%m-%d').date()
        end_date = datetime.strptime(end_date, '%Y-%m-%d').date()
    else:
        raise ValueError("Invalid date_query format.")
    return start_date.isoformat(), end_date.isoformat()

# Example usage for LangChain tools:
def main():
    db = SessionLocal()

    # Electricity Price Tool
    electricity_tool = ElectricityPriceTool(db)
    electricity_request = {
        "city": "Lillehammer",
        "state": "Oslo",
        "date_query": "2024-01-01/2024-01-01"
    }
    electricity_data = electricity_tool.get_electricity_prices(electricity_request)
    print(json.dumps(electricity_data, indent=4))

    # Weather Data Tool
    weather_tool = WeatherDataTool(db)
    weather_request = {
        "city": "Lillehammer",
        "state": "",
        "date_query": "2024-07-23/2024-08-06"
    }
    weather_data = weather_tool.get_weather_data(weather_request)
    print(json.dumps(weather_data, indent=4))

if __name__ == "__main__":
    main()

[
    {
        "NOK_per_kWh": 0.60463,
        "EUR_per_kWh": 0.05379,
        "EXR": 11.2405,
        "time_start": "2024-01-01T00:00:00+01:00",
        "time_end": "2024-01-01T01:00:00+01:00"
    },
    {
        "NOK_per_kWh": 0.55056,
        "EUR_per_kWh": 0.04898,
        "EXR": 11.2405,
        "time_start": "2024-01-01T01:00:00+01:00",
        "time_end": "2024-01-01T02:00:00+01:00"
    },
    {
        "NOK_per_kWh": 0.30855,
        "EUR_per_kWh": 0.02745,
        "EXR": 11.2405,
        "time_start": "2024-01-01T02:00:00+01:00",
        "time_end": "2024-01-01T03:00:00+01:00"
    },
    {
        "NOK_per_kWh": 0.27517,
        "EUR_per_kWh": 0.02448,
        "EXR": 11.2405,
        "time_start": "2024-01-01T03:00:00+01:00",
        "time_end": "2024-01-01T04:00:00+01:00"
    },
    {
        "NOK_per_kWh": 0.26988,
        "EUR_per_kWh": 0.02401,
        "EXR": 11.2405,
        "time_start": "2024-01-01T04:00:00+01:00",
        "time_end": "2024-01-01T05:00:00+01:00"
    }

In [ ]:
tools = 